In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load dataset
df = pd.read_csv("data/ratings.csv")
df.head()

In [ ]:
# Pivot ratings into a user-item matrix
user_item_matrix = df.pivot_table(index="userId", columns="movieId", values="rating")
user_item_matrix.fillna(0, inplace=True)
user_item_matrix.head()

In [ ]:
# Compute user-user similarity
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, 
                                  index=user_item_matrix.index, 
                                  columns=user_item_matrix.index)
user_similarity_df.head()

In [ ]:
def recommend_movies(user_id, num_recommendations=5):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)
    similar_users = similar_users.drop(user_id)
    top_users = similar_users.head(3).index
    user_movies = set(df[df["userId"] == user_id]["movieId"])
    recommendations = df[df["userId"].isin(top_users)].groupby("movieId")["rating"].mean().sort_values(ascending=False)
    recommendations = recommendations[~recommendations.index.isin(user_movies)]
    return recommendations.head(num_recommendations)

recommend_movies(1, 5)

In [ ]:
# Item-item similarity
item_similarity = cosine_similarity(user_item_matrix.T)
item_similarity_df = pd.DataFrame(item_similarity, 
                                  index=user_item_matrix.columns, 
                                  columns=user_item_matrix.columns)
item_similarity_df.head()

In [ ]:
def recommend_movies_item(movie_id, num_recommendations=5):
    similar_items = item_similarity_df[movie_id].sort_values(ascending=False)
    similar_items = similar_items.drop(movie_id)
    return similar_items.head(num_recommendations)

recommend_movies_item(1, 5)

In [ ]:
user_similarity_df.to_csv("user_similarity.csv")
item_similarity_df.to_csv("item_similarity.csv")
print("Similarity matrices saved successfully.")